In [3]:
%%bash

echo $(pwd)
ls -lah 


/Users/dsuveges/project/random_notebooks/issue-2607_consolidating_OTAR_tables
total 592
drwxr-xr-x   6 dsuveges  384566875   192B 31 May 16:51 .
drwxrwxr-x  84 dsuveges  384566875   2.6K 31 May 16:31 ..
drwxr-xr-x   3 dsuveges  384566875    96B 31 May 16:51 .ipynb_checkpoints
-rw-r--r--@  1 dsuveges  384566875    87K 31 May 16:38 EFO Spreadsheet.xlsx
-rw-r--r--@  1 dsuveges  384566875   202K 31 May 16:37 OTAR project EFO mappings for disease profile pages.xlsx
-rw-r--r--   1 dsuveges  384566875   605B 31 May 16:34 consolidating tables.ipynb


# Collate files.

1. Read both files, master sheet, try to find differences.
2. Make sure the proper updates are selected and generate a table, group by study etc.
3. Save xls file with two sheets.

In [127]:
import pandas as pd


master = pd.read_excel('OTAR project EFO mappings for disease profile pages.xlsx', sheet_name='master')
project2disease = pd.read_excel('OTAR project EFO mappings for disease profile pages.xlsx', sheet_name='Project2Disease')
meta = pd.read_excel('OTAR project EFO mappings for disease profile pages.xlsx', sheet_name='MetaData')


print(len(master))
print(len(project2disease))
print(len(meta))


273
72
81


In [41]:
for table in [master, project2disease, meta]:
    print(len(table.otar_code.unique()))
    
print('so some of the project to disease mapping is missing. Could be projects with no disease? Shouldn\'t be null?')

(
    meta
    .loc[ ~meta.otar_code.isin(project2disease.otar_code.to_list())]
)

79
72
79
so some of the project to disease mapping is missing. Could be projects with no disease? Shouldn't be null?


,otar_code,project_name,project_lead,generates_data,data_available,project_status,open_targets_therapeutic_area,efo_disease_label,efo_disease_id,pubmed_ids,epmc_preprint_ids,other_publication_dois,notes
73,OTAR2076,Phenotypic mapping of gene functions in immune...,Gosia Trynka,Y,Not yet available,Active,Immunology / Inflammation,colorectal cancer,EFO_0005842,NaN,NaN,NaN,NaN
74,OTAR2079,Saturation Genome Editing in Immune Genes,Gosia Trynka/Dave Adams,Y,Not yet available,Active,Immunology / Inflammation,Rheumatoid arthritis,EFO_0000685,NaN,NaN,NaN,NaN
75,OTAR2080,Natural Killer cell evasion and exhaustion med...,Annie Speak,Y,Not yet available,Active,Oncology,cancer,EFO_0000311,NaN,NaN,NaN,NaN
77,OTAR2075,Loss-of-function,Maya Ghoussaini/Gosya Trynka,Y,Not yet available,Active,Informatics,NaN,NaN,NaN,NaN,NaN,NaN
78,OTAR2077,eQTL Catalogue expansion,Kaur Alasoo,Y,Not yet available,Active,Informatics,NaN,NaN,NaN,NaN,NaN,NaN
79,OTAR2078,Target Engine,Ian Dunham,Y,Not yet available,Active,Informatics,NaN,NaN,NaN,NaN,NaN,NaN
80,OTAR2081,Sequence variation,Pedro Beltrao,Y,Not yet available,Active,Informatics,NaN,NaN,NaN,NaN,NaN,NaN


There are 7 projects. 3 of them HAS disease. We need to add them.

In [129]:
# These are the projects from the meta table that were not found in the project to disease table:
new_projects = (
    meta
    .loc[ ~meta.otar_code.isin(project2disease.otar_code.to_list())]
    [['otar_code', 'efo_disease_id']]
    .drop_duplicates()
)


# Combining dataframes:
project2disease_updated = (

    # Adding new disease/project pairs
    pd.concat([project2disease, new_projects])
)

# updating EFO mapping for OTAR036:
(
    project2disease_updated
    .loc[project2disease_updated.otar_code == 'OTAR036', 'efo_disease_id']
) = 'Orphanet_282|MONDO_0005180|MONDO_0004975'


Do we have all the diseases? If we split the disease column and explode, we should get the same number of rows in the master table.... I would be super surprised if that would be the case.

In [130]:
(
    project2disease_updated
    .fillna('-')
    .assign(diseases = lambda df: df.efo_disease_id.str.split('|'))
    .explode('diseases')
)

,otar_code,efo_disease_id,diseases
0,OTAR001,-,-
1,OTAR003,-,-
2,OTAR004,-,-
3,OTAR005,-,-
4,OTAR006,-,-
...,...,...,...
75,OTAR2080,EFO_0000311,EFO_0000311
77,OTAR2075,-,-
78,OTAR2077,-,-
79,OTAR2078,-,-


In [58]:
print('Columns not found in meta:')
print([column for column in master.columns if column not in meta.columns])

print('Columns not found in master:')
print([column for column in meta.columns if column not in master.columns])

Columns not found in meta:
[]
Columns not found in master:
[]


In [63]:
# Dropping disease columns from meta table:
meta = meta.drop(['efo_disease_label', 'efo_disease_id'], axis=1)
print(len(meta))
# 

master = master.drop(['efo_disease_label', 'efo_disease_id'], axis=1).drop_duplicates()
print(len(master))

81
91


In [92]:
for otar_code in sorted(master['otar_code'].value_counts().loc[lambda s: s > 1].index.to_list()):
    print('\n' + otar_code)
    for c in master.columns:
        if len(master.query('otar_code == @otar_code')[c].unique()) > 1:
            print(c + ': ' + ','.join([str(x) for x in master.query('otar_code == @otar_code')[c].unique()]))


OTAR015
other_publication_dois: nan, 
notes: * Mapping file generated by MS and reviewed by project team,nan

OTAR016
notes: GDSC cell lines are in OTAR015 above so this is dealt with already. The analysis gets lumped into the types above,nan

OTAR033
notes: Also "atopic dermatitis" aka "atopic eczema" EFO_0000274,nan

OTAR037
pubmed_ids: 33152068,>>
notes: Also Parkinson's Disease MONDO_0005180,nan

OTAR040
pubmed_ids: 31548716|32286271 | 34402426,>>
epmc_preprint_ids: PPR380389 ,nan
notes: nan,No! Inflammatory Bowel Disease EFO_0003767

OTAR041
project_name: CELLector,CELLector 
pubmed_ids: 32437684,>>
notes: is cancer too broad of a term?,nan

OTAR2055
pubmed_ids: 31862961|33712601 | 33500573,>>
epmc_preprint_ids: PPR166364,>>
notes: use same mapping file used for OTAR015? YES,nan

OTAR2059
notes: Oncology and neurodegeneration. Get oncology types from Panos ,nan

OTAR2063
notes: nan,No! Inflammatory Bowel Disease EFO_0003767


### Updating fields in the metadata sheet

- [x] OTAR015: Nothing to update
- [x] OTAR016: Nothing to update
- [x] OTAR033: EFO checked, but eczema is there, so note is deleted.
- [x] OTAR037: EFO checked, but Parkinson's disease is there, so note is deleted.
- [x] OTAR040: No update was necessary.
- [x] OTAR041: No update
- [x] OTAR2055: No update
- [x] OTAR2059: No update
- [x] OTAR2063: No update

Index(['otar_code', 'project_name', 'project_lead', 'generates_data',
       'data_available', 'project_status', 'open_targets_therapeutic_area',
       'pubmed_ids', 'epmc_preprint_ids', 'other_publication_dois', 'notes'],
      dtype='object')

In [131]:
(
    project2disease_updated
    .loc[(project2disease_updated.otar_code == 'OTAR2063') ]
    .efo_disease_id
    .to_list()
)

['EFO_0000685|MONDO_0005301|EFO_0003767|MONDO_0004979|MONDO_0005271']

In [132]:
import numpy as np

def clean_list(s: str) -> str:
    """This function assumes to get a list of string values concatenated using |
    
    Removes all spaces, trims, if empty string is given, returns none.
    """
    
    if (s == '') or (s is None) or (s == np.nan) or (isinstance(s, float)):
        return None

    return s.replace(' ', '')

# Cleaning disease labels:
project2disease_updated = (
    project2disease_updated
    .assign(efo_disease_id = lambda df: df.efo_disease_id.apply(clean_list))
)

In [122]:
(
    project2disease_updated
    .loc[(~project2disease_updated.efo_disease_id.isna())]
    .loc[lambda df: ~(df.efo_disease_id.str.contains('_'))]
)

In [121]:
# Cleaning the list in the meta dataset:
meta = (
    meta
    .assign(
        other_publication_dois = lambda df: df.other_publication_dois.apply(clean_list),
        epmc_preprint_ids = lambda df: df.epmc_preprint_ids.apply(clean_list),
        pubmed_ids = lambda df: df.pubmed_ids.apply(clean_list)
    )
)

### Saving data

The two datasets are saved as excel files. The data is then transferred to the original destination, then a new release is saved.

In [123]:
meta.to_excel('meta_table_updated.xlsx', index=False)

In [152]:
project2disease_updated = (
    project2disease_updated
    .assign(efo_disease_id = lambda df: df.efo_disease_id.str.split('|'))
    .explode('efo_disease_id')
#     .to_excel('project2disease_updated.xlsx', index=False)
)
project2disease_updated.to_excel('project2disease_updated.xlsx', index=False)

In [135]:
len(project2disease_updated.otar_code.unique())

79

In [137]:
meta.otar_code.value_counts().loc[lambda s: s > 1]

OTAR2059    3
Name: otar_code, dtype: int64

In [140]:
meta.query("otar_code == 'OTAR2059'")

,otar_code,project_name,project_lead,generates_data,data_available,project_status,open_targets_therapeutic_area,efo_disease_label,efo_disease_id,pubmed_ids,epmc_preprint_ids,other_publication_dois,notes
56,OTAR2059,Experimental Platform Pilot,Panos Zalmas,Y,Does not generate data,Active,Oncology,breast carcinoma,EFO_0000305,--,NaN,NaN,Oncology and neurodegeneration. Get oncology t...
57,OTAR2059,Experimental Platform Pilot,Panos Zalmas,Y,Does not generate data,Active,Oncology,colorectal carcinoma,EFO_1001951,--,NaN,NaN,NaN
76,OTAR2059,Experimental Platform Pilot\n,Panos Zalmas/Mathew Garnett,Y,Not yet available,Active,Other Therapy Area,NaN,NaN,NaN,NaN,NaN,NaN
